In [1]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.quant = torch.ao.quantization.QuantStub()
        self.model = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=2),
            nn.ReLU(),
            nn.Conv2d(8, 8*8, 3, stride=2),
            nn.ReLU(),
            nn.AvgPool2d(3, 2),
            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(256, 128), 
            nn.Dropout(),
            nn.Linear(128, 10) 
            # nn.Softmax()
        )
        self.dequant = torch.ao.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        y = self.model(x)
        return self.dequant(y)
    

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to("cuda")
        y = y.to("cuda")
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [3]:
model = Classifier()

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
loss_fn = nn.CrossEntropyLoss()

In [4]:
model = model.to("cuda")
for i in range(100):
    train(train_dataloader, model, loss_fn, optimizer)
model = model.to("cpu")


loss: 2.296914  [   64/60000]
loss: 2.291912  [ 6464/60000]
loss: 2.294836  [12864/60000]
loss: 2.288554  [19264/60000]
loss: 2.309775  [25664/60000]
loss: 2.286273  [32064/60000]
loss: 2.322832  [38464/60000]
loss: 2.305559  [44864/60000]
loss: 2.295563  [51264/60000]
loss: 2.307890  [57664/60000]
loss: 2.297695  [   64/60000]
loss: 2.284710  [ 6464/60000]
loss: 2.295141  [12864/60000]
loss: 2.285764  [19264/60000]
loss: 2.301398  [25664/60000]
loss: 2.281679  [32064/60000]
loss: 2.315851  [38464/60000]
loss: 2.290140  [44864/60000]
loss: 2.298728  [51264/60000]
loss: 2.291737  [57664/60000]
loss: 2.282570  [   64/60000]
loss: 2.296183  [ 6464/60000]
loss: 2.305233  [12864/60000]
loss: 2.283379  [19264/60000]
loss: 2.302626  [25664/60000]
loss: 2.267376  [32064/60000]
loss: 2.306221  [38464/60000]
loss: 2.285654  [44864/60000]
loss: 2.290928  [51264/60000]
loss: 2.281875  [57664/60000]
loss: 2.292846  [   64/60000]
loss: 2.284919  [ 6464/60000]
loss: 2.280387  [12864/60000]
loss: 2.26

In [5]:
%%timeit
test(test_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 74.5%, Avg loss: 0.703184 

Test Error: 
 Accuracy: 74.5%, Avg loss: 0.703184 

Test Error: 
 Accuracy: 74.5%, Avg loss: 0.703184 

Test Error: 
 Accuracy: 74.5%, Avg loss: 0.703184 

Test Error: 
 Accuracy: 74.5%, Avg loss: 0.703184 

Test Error: 
 Accuracy: 74.5%, Avg loss: 0.703184 

Test Error: 
 Accuracy: 74.5%, Avg loss: 0.703184 

Test Error: 
 Accuracy: 74.5%, Avg loss: 0.703184 

1.26 s ± 66.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
model = model.to("cpu")

model.eval()
backend = "x86"
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend

model_static_quantized = torch.quantization.prepare(model, inplace=False)
model_static_quantized = torch.quantization.convert(model_static_quantized, inplace=False)

/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/observer.py:1272: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [7]:
%%timeit
test(test_dataloader, model_static_quantized, loss_fn)

Test Error: 
 Accuracy: 61.5%, Avg loss: 1.097411 

Test Error: 
 Accuracy: 61.5%, Avg loss: 1.097411 

Test Error: 
 Accuracy: 61.5%, Avg loss: 1.097411 

Test Error: 
 Accuracy: 61.5%, Avg loss: 1.097411 

Test Error: 
 Accuracy: 61.5%, Avg loss: 1.097411 

Test Error: 
 Accuracy: 61.5%, Avg loss: 1.097411 

Test Error: 
 Accuracy: 61.5%, Avg loss: 1.097411 

Test Error: 
 Accuracy: 61.5%, Avg loss: 1.097411 

885 ms ± 400 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
torch.save(model_static_quantized, "model_post_static_quantized.pt")